In [ ]:
#import some necessary librairies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import plotly.express as px
from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output


In [ ]:
#Dataset: https://www.kaggle.com/muhammetvarl/laptop-price
df = pd.read_csv('../input/laptop-price/laptop_price.csv',encoding='latin-1')
df.head()

In [ ]:
df.info()

# Data Visualization and Feature Engineering

In [ ]:
df['Inches'] = df['Inches'].apply(str)
df['Weight'] = df['Weight'].apply(lambda x: str(x).split('k')[0])

In [ ]:
df['Weight'] = df['Weight'].apply(float)

In [ ]:
temp = df.groupby(['Company']).mean().reset_index().sort_values(by='Price_euros',ascending=False)
plt.figure(figsize = (20,20))
splot = sns.barplot(data = temp,x = 'Company', y = 'Price_euros',palette = 'RdBu')
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.xlabel("Company", size=14)
plt.ylabel("Price", size=14)
plt.title('Average Laptop Price Per Company')

In [ ]:
temp = df.groupby(['Company','Ram']).mean().reset_index().sort_values(by='Price_euros',ascending=False)
px.scatter(data_frame = temp, x = 'Company', y = 'Price_euros',color = 'Ram',title = 'Average Laptop Price for Each Company/RAM amount')

In [ ]:
px.scatter(data_frame = df,x = 'Inches',y = 'Price_euros')

In [ ]:
px.scatter(data_frame = df,x = 'Ram',y = 'Price_euros')

In [ ]:
#Removing Outliers after checking them using scatter plot
df = df[~((df['Price_euros'] > 3500) &(df['Ram']== '8GB'))]
df = df[~((df['Price_euros'] > 3500) &(df['Ram']== '16GB'))]

In [ ]:
df = df[~((df['Price_euros'] == 4899) &(df['Inches']== 15.6))]
df = df[~((df['Price_euros'] >= 5000) &(df['Inches']== 17.3))]

> We can easily compare laptops prices with respect to ram with this amazing graph above

In [ ]:
temp = df.groupby(['Company','Memory']).mean().reset_index().sort_values(by='Price_euros',ascending=False)
px.scatter(data_frame = temp, x = 'Memory', y = 'Price_euros',color = 'Company',title = 'Average Laptop Price for Each Company/Memory')

In [ ]:
temp = df.groupby(['Company','Inches']).mean().reset_index().sort_values(by='Price_euros',ascending=False)
px.scatter(data_frame = temp, x = 'Inches', y = 'Price_euros',color = 'Company', title = 'Average Laptop Price for Each Company/Size in Inches')

In [ ]:
sns.distplot(df['Price_euros'] , fit=norm)
# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df['Price_euros'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price_euros distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(df['Price_euros'], plot=plt)
plt.show()

**Log Transformation**

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
df["Price_euros"] = np.log1p(df["Price_euros"])

#Check the new distribution 
sns.distplot(df['Price_euros'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df['Price_euros'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price_euros distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(df['Price_euros'], plot=plt)
plt.show()

In [ ]:
#Correlation map to see how features are correlated with Price_euros
corrmat = df.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True,cmap = 'RdBu',annot = True)

**There is no Correlation between the price and other variables and there is no intercorrelation**

In [ ]:
#Dont need ID column for the model
df.drop(columns = 'laptop_ID',inplace = True)

In [ ]:
df.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ['Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu',
       'Ram', 'Memory', 'Gpu', 'OpSys']
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))


In [ ]:
df.head()

# Model Building

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
X = df.iloc[:,:-1]
y = df['Price_euros']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

**Lasso Regression**

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**Elastic Net Regression**

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**Kernel Ridge Regression**

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**Gradient Boosting Regression**

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**XGBoost Regression**

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**LGBM**

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

# Fitting the best model and getting the predictions array

In [ ]:
GBoost.fit(X_train,y_train)
gboost_pred = GBoost.predict(X_test)


In [ ]:
result = pd.DataFrame({'Y_TEST':y_test.values,'Y_PRED' : pd.Series(gboost_pred).values})

In [ ]:
px.line(data_frame = result)

# Almost perfect fit on the test data!